In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    'save_best': False,
    'device': "cuda:0"
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_train", ENTITY_NAME: "obj_detect_inference_files"} # stored detector files from train step
    ],
    
    tmp_entities =
    [
        { ENTITY_NAME: "obj_detect_inference_files" }, # temporary detector files from train step
        { ENTITY_NAME: "obj_detect_onnx_files"}, # temporary detector onnx files after converting 
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" } # stored BentoService
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# specify all notebook wide libraries imports here
# Sinara lib imports is left in the place of their usage
import json
import os
import os.path as osp
import glob
import torch
import numpy as np
import onnxruntime, pickle, shutil

from mmengine.config import Config as MmConfig
from mmdeploy.backend.sdk.export_info import export2SDK
from mmdeploy.apis import torch2onnx 

In [ ]:
# run spark
from sinara.spark import SinaraSpark
from sinara.archive import SinaraArchive

spark = SinaraSpark.run_session(0)
archive = SinaraArchive(spark)
SinaraSpark.ui_url()

### Loading obj_detect_inference_files from the model_train step 
(weights, configs, test image)

In [ ]:
inputs = substep.inputs(step_name = "model_train")
tmp_entities = substep.tmp_entities()
# copy config from previos step to outputs

archive.unpack_files_from_store_to_tmp(store_path=inputs.obj_detect_inference_files, tmp_entity_dir=tmp_entities.obj_detect_inference_files)

### Select obj_detector weights for converting

In [ ]:
# Selecting a weights file to convert to onnx format (best, latest epoch, etc.)

best_weights_pths = glob.glob(f"{tmp_entities.obj_detect_inference_files}/*best*")
latest_weights_pths = glob.glob(f"{tmp_entities.obj_detect_inference_files}/*latest*")

weights_pths = best_weights_pths if substep_params['save_best'] and len(best_weights_pths) > 0 else latest_weights_pths

weights_pths.sort(key=lambda file: osp.getmtime(file))

selected_weights_pth = weights_pths[-1]
mmengine_cfg_path = osp.join(tmp_entities.obj_detect_inference_files, "last_cfg.py")
mmengine_cfg = MmConfig.fromfile(mmengine_cfg_path)

### Export to ONNX

#### Preparing obj_detector weights for export

In [ ]:
# Clean model weigths - delete data of optimizer

state_dict = torch.load(selected_weights_pth, map_location=torch.device('cpu'))

print(f"{state_dict.keys()=}")
if "optimizer" in state_dict:
    del state_dict["optimizer"]
    
clean_weigths_pth = osp.splitext(selected_weights_pth)[0]+"_clean.pth"
torch.save(state_dict, clean_weigths_pth)

#### Setting up basic model training mmdeploy config parameters

In [ ]:
export_params = step_params["export_params"]

mmdeploy_cfg_path = osp.join(osp.expanduser("~"), 'mmdeploy', 'configs', 'mmdet', 'detection', 'detection_onnxruntime_dynamic.py')
mmdeploy_cfg = MmConfig.fromfile(mmdeploy_cfg_path)

# set converting onnx file name
mmdeploy_cfg.save_file = os.path.basename(clean_weigths_pth).split(".")[0] + ".onnx"
# set parameters NMS for export to onnx 
mmdeploy_cfg.codebase_config.post_processing.iou_threshold = export_params["iou_threshold"]
mmdeploy_cfg.codebase_config.post_processing.keep_top_k = export_params["keep_top_k"]
mmdeploy_cfg.codebase_config.post_processing.max_output_boxes_per_class = export_params["max_output_boxes_per_class"]
mmdeploy_cfg.codebase_config.post_processing.pre_top_k = export_params["pre_top_k"]
mmdeploy_cfg.codebase_config.post_processing.score_threshold = export_params["score_threshold"]

# output information for mmdeploy
export2SDK( deploy_cfg=mmdeploy_cfg,
            model_cfg=mmengine_cfg,
            work_dir=tmp_entities.obj_detect_onnx_files,
            pth=clean_weigths_pth,
            device=substep_params['device'])

# reopen config of model
# fix bug in mmdeploy: after export2SDK changes model_cfg with error (use attributes size in augmentation, use Collect in preprocessing)
mmengine_cfg = MmConfig.fromfile(mmengine_cfg_path)

#### Converting to onnx file

In [ ]:
dummy_img = np.zeros(list(mmengine_cfg.img_size)+[3], dtype=np.uint8)  # image zeros by shape [width, height, chanels]

torch2onnx(img=dummy_img,
           work_dir=tmp_entities.obj_detect_onnx_files,
           save_file=mmdeploy_cfg.save_file,
           deploy_cfg=mmdeploy_cfg,
           model_cfg=mmengine_cfg,
           model_checkpoint=clean_weigths_pth,
           device=substep_params['device'])

### Pack to REST BentoService

In [ ]:
from bento_service import ModelService
from pre_post_processing import PrePostProcessing

CLASSES = mmengine_cfg.metainfo.CLASSES
INPUT_SIZE = mmengine_cfg.img_size
CATEGORIES = [{"id": class_id+1, "name": class_name} for class_id, class_name in enumerate(CLASSES)]

outputs = substep.outputs()

# copy test image 
test_image_path = osp.join(tmp_entities.obj_detect_inference_files, "test.jpg")
onnx_test_image_path =  osp.join(tmp_entities.obj_detect_onnx_files, "test.jpg")
shutil.copy(test_image_path, onnx_test_image_path)
assert osp.exists(onnx_test_image_path)

# inicialize onnx model
onnx_file = os.path.join(tmp_entities.obj_detect_onnx_files, mmdeploy_cfg.save_file)
assert osp.exists(onnx_file)
ort_session = onnxruntime.InferenceSession(onnx_file, providers=['CPUExecutionProvider'])
input_name = ort_session.get_inputs()[0].name
output_name = [out.name for out in ort_session.get_outputs()]

# read test image and processing for inference by onnx
pre_post_processing = PrePostProcessing()
input_data, scale_factors, img_ori_size = pre_post_processing.prep_processing(onnx_test_image_path, input_size=INPUT_SIZE)

# inference onnx by test image
outs = ort_session.run(output_name, {input_name: input_data})
outs = pre_post_processing.post_processing(outs, scale_factors, img_ori_size, categories=CATEGORIES)

# save and reopen pickle file output of inference by test image
with open(osp.join(tmp_entities.obj_detect_onnx_files, 'test_result.pkl'), 'wb') as pkl_file:
    pickle.dump(outs, pkl_file)    
with open(osp.join(tmp_entities.obj_detect_onnx_files, 'test_result.pkl'), 'rb') as f_id:
    test_result = f_id.read()    
    
# open test image
with open(onnx_test_image_path, 'rb') as f_id:
    test_image = f_id.read()   

#### Packaging obj_detector files to bento_service artifacts

In [ ]:
model_service = ModelService()
model_service.pack('model', onnx_file)
model_service.pack('test_image', test_image)
model_service.pack('test_result', test_result)    
model_service.pack('categories', CATEGORIES)
model_service.pack('input_size', INPUT_SIZE)

### Send packaged onnx_obj_detector to outputs

In [ ]:
# save model as a bento pack
from sinara.bentoml import save_bentoservice
save_bentoservice(model_service, path=outputs.bento_service, substep=substep)

In [ ]:
# stop spark
SinaraSpark.stop_session()